# 1 - Setting up

Import relevant libraries.

In [21]:
import json
import pandas as pd
import requests

Define all configuration variables.

In [22]:
URL = 'https://exams.is.ed.ac.uk/search/'
FILENAME = 'data.json'

# 2 - Retrieving and cleaning data

Retrieve examination diet data and save the output to `data.json`.

In [25]:
with requests.Session() as s:
    raw_data = s.get(URL)
    with open(FILENAME, 'w') as f:
        d_list = json.loads(raw_data.text)['data']
        f.write(json.dumps(d_list))

Convert the list of dictionaries to a data frame and drop irrelevant columns.

In [43]:
with open(FILENAME, 'r') as f:
    d_list = json.loads(f.read())
    
df = pd.DataFrame(d_list)
df['course_code'], df['course_name'] = df['title'].str.split(' - ', 1).str
df['start_date'] = df['start_date'].dt


df.drop(columns=['hours_since_change', 'last_changed_date', 'last_changed_text', 'title'], inplace=True)

df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 8 columns):
duration        991 non-null object
end_time        991 non-null object
start_date      991 non-null object
start_time      991 non-null object
subject_area    991 non-null object
venues          991 non-null object
course_code     991 non-null object
course_name     991 non-null object
dtypes: object(8)
memory usage: 62.0+ KB


,duration,end_time,start_date,start_time,subject_area,venues,course_code,course_name
0,2:00,4:30 p.m.,"Monday, 13th May 2019",2:30 p.m.,Accounting,[{'location': 'The Pleasance Sports Hall'}],ACCN08008,Accountancy 1B
1,2:00,4:30 p.m.,"Tuesday, 30th April 2019",2:30 p.m.,Accounting,[{'location': 'Patersons Land Room G1 (Enter ...,ACCN08010,Accountancy 2B
2,2:00,11:30 a.m.,"Wednesday, 15th May 2019",9:30 a.m.,Accounting,[{'location': 'Playfair Library'}],ACCN10002,Advanced Management Accounting
3,2:00,4:30 p.m.,"Tuesday, 7th May 2019",2:30 p.m.,Accounting,[{'location': 'Patersons Land Room 1.26 (Ent...,ACCN10005,Audit Practice
4,3:00,12:30 p.m.,"Friday, 3rd May 2019",9:30 a.m.,Accounting,[{'location': 'McEwan Hall - Foyer Room 3 & 4 ...,ACCN10010,Management Accounting Applications
5,2:00,11:30 a.m.,"Thursday, 9th May 2019",9:30 a.m.,Accounting,[{'location': 'Chrystal MacMillan Seminar Room...,ACCN10014,Accounting and Everyday Life
6,2:00,4:30 p.m.,"Friday, 17th May 2019",2:30 p.m.,African Studies,[{'location': 'Patersons Land Room 1.26 (Ent...,AFRI08003,Introductory Swahili Language and Culture 1B -...
7,2:00,4:30 p.m.,"Monday, 20th May 2019",2:30 p.m.,African Studies,[{'location': 'The Pleasance Sports Hall'}],AFRI08008,Africa in the Contemporary World
8,2:00,4:30 p.m.,"Monday, 6th May 2019",2:30 p.m.,Ancient History,[{'location': 'Appleton Tower Concourse'}],ANHI08013,Ancient History 2b: Themes and Theories in Anc...
9,2:00,4:30 p.m.,"Thursday, 9th May 2019",2:30 p.m.,Ancient History,[{'location': 'Patersons Land Room G1 (Enter ...,ANHI10011,Roman Slavery
